## Value at Risk (VaR)

Value at Risk (VaR) tells the maximum possible loss for a given portfolio over a certain time horizon, with a certain confidence level $\alpha$, or
$$
\Pr\bigl(\text{Loss}\le -\text{VaR}\bigr)=1-\alpha
$$
Consider a portfolio of n assets with corresponding weights $w$ and covariance matrix $\sigma$, then the value this portfolio V, and its return and risk are
$$
\begin{aligned}
V          &= \sum_{i=1}^{n} V_i \\
r_p        &= w^{T}r \\
\sigma_p^2 &= w^{T}\Sigma w
\end{aligned}
$$
Assuming return normality, The value at risk of the portfolio in dollar amount is
$$
\text{VaR}_p = z_\alpha\,\sigma_p\,V
$$
where z is the inverser normal of confidence level $\alpha$.

Marginal VaR is defined as derivative of portfolio w.r.t the per unit dollar change in value of the component stock, or
$$
\text{VaR}_i = z_\alpha\,\sigma_i\,V_i,
\qquad
\text{VaR}_p < \sum_{i=1}^{n}\text{VaR}_i
$$
It also can be rewritten as
$$
\text{MVaR}_i
= z_\alpha\,\frac{\sigma_{ip}}{\sigma_p}
= z_\alpha\,\sigma_i\,\rho_{ip}
= z_\alpha\,\sigma_p\,\beta_{ip}
$$
At last, Component VaR (CVaR) is defined as,
$$
\text{CVaR}_i = \text{MVaR}_i \; \times V_i
$$
$$
\sum_{i=1}^{n}\text{CVaR}_i \;=\; \text{VaR}_p
$$

In [4]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm

assets   = ['AAPL', 'KO', 'DIS', 'XOM', 'JPM', 'MCD', 'WMT']
holdings = np.array([100, 200, 300, 400, 500, 600, 700])   # shares
start, end = '2015-01-01', '2017-12-31'
alpha   = 0.95                           # confidence level
z_alpha = norm.ppf(alpha)                # ≈ 1.645 for 95 %

# --------------------------------------------------
# 1 )  get prices with yf.Ticker() one-by-one
# --------------------------------------------------
price_dict = {}
for tic in assets:
    price_dict[tic] = (
        yf.Ticker(tic)
          .history(start=start, end=end)["Close"]
          .rename(tic)                   # nice column name
    )

prices = pd.concat(price_dict.values(), axis=1)            # (T × n)

# --------------------------------------------------
# 2 )  daily log-returns, covariance
# --------------------------------------------------
log_ret = np.log(prices / prices.shift(1)).dropna()
cov_mat = log_ret.cov().values                # Σ   (n×n)

# --------------------------------------------------
# 3 )  current dollar exposure
# --------------------------------------------------
latest_px = prices.iloc[-1].values            # price on 'end' date
V_i = latest_px * holdings                    # $ exposure vector (n,)
V_p = V_i.sum()                               # total $

# --------------------------------------------------
# 4 )  portfolio VaR (delta-normal)
# --------------------------------------------------
sigma_p = np.sqrt(V_i @ cov_mat @ V_i)        # σₚ   ($ units)
VaR_p   = z_alpha * sigma_p                   # in $

# --------------------------------------------------
# 5 )  individual VaR, Marginal VaR, Component VaR
# --------------------------------------------------
sigma_i = np.sqrt(np.diag(cov_mat))           # σᵢ   (% units)
VaR_i   = z_alpha * sigma_i * V_i             # standalone VaR ($)

cov_ip  = cov_mat @ V_i                       # cov(asset, p) in $²
beta_i  = cov_ip / (sigma_p**2 / V_p)         # portfolio beta
MVaR_i  = (VaR_p / V_p) * beta_i              # $/per extra $

CVaR_i  = MVaR_i * V_i                        # Component VaR ($)

# --------------------------------------------------
# 6 )  tidy table
# --------------------------------------------------
table = pd.DataFrame({
    "Position ($)" : V_i,
    "Position (%)" : V_i / V_p,
    "CVaR ($)"     : CVaR_i,
    "CVaR (%)"     : CVaR_i / VaR_p,
    "Beta"         : beta_i
}, index=assets).round(4)

print(f"\n=== Portfolio VaR (α={alpha:.0%}) : ${VaR_p:,.2f} ===")
print(table)



=== Portfolio VaR (α=95%) : $2,813.27 ===
      Position ($)  Position (%)   CVaR ($)  CVaR (%)    Beta
AAPL     3971.5733        0.0184    45.2053    0.0161  0.8733
KO       7257.0068        0.0336    50.8528    0.0181  0.5376
DIS     30851.8822        0.1429   365.6325    0.1300  0.9092
XOM     23623.4879        0.1095   276.3547    0.0982  0.8975
JPM     43168.3044        0.2000   732.1264    0.2602  1.3012
MCD     86602.9907        0.4012  1157.5416    0.4115  1.0255
WMT     20361.7090        0.0943   185.5606    0.0660  0.6992
